# Классификатор на основе нейронной сети

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.models import load_model
import numpy as np
import pandas as pd

In [17]:
# Загружаем файл с разделителем ';' в DF
df = pd.read_csv(fr'nn_features_and_target.csv', delimiter=';')
# df.columns = df.columns.astype(str)
df['29'] = df.apply(lambda x: 1.0 if x['28'] == 0.0 else 0.0, axis=1)
print(df.to_string(max_rows=8, max_cols=30))

             0         1         2         3         4         5         6         7         8         9        10        11        12        13        14        15        16        17        18        19        20   21   22   23   24   25   26   27   28   29
0     1.000000  0.949765  0.946863  0.890311  0.886746  0.810817  0.768762  0.702327  0.658250  0.519462  0.000000  0.071725  0.156965  0.237875  0.342755  0.385964  0.561642  0.618664  0.710416  0.721658  0.876098  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
1     1.000000  0.999613  0.949249  0.946274  0.888331  0.884700  0.808215  0.764843  0.699029  0.650257  0.510770  0.000000  0.073884  0.163346  0.246354  0.359640  0.402954  0.585638  0.643909  0.737880  0.749382  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
2     0.879807  0.815902  0.811820  0.731742  0.685201  0.616099  0.559507  0.401556  0.216736  0.112487  0.000000  0.101511  0.235431  0.280920  0.482243  0.544868  0.668686  0.686063  0.859401  0.888299  1.000000  0.0 

In [18]:
# Преобразование в DataSet
dataset = df.values  # Dataframe преобразуем в Dataset для Keras
dataset = dataset.astype(np.float32)  # Смена типа для корректной работы Keras
# Все, что стоит перед запятой, относится к строкам массива, а все, что стоит после запятой,
# относится к столбцам массивов.
X = dataset[:, 0:21]  # Срез массива по фичам
y = dataset[:, 28:30]  # Срез массива по labels

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.15,
    random_state=3,
    shuffle=True
)

# # scaler = StandardScaler()
# # scaler = MinMaxScaler() 
# scaler = MinMaxScaler(feature_range=(-1, 1))
# X_train = scaler.fit_transform(X_train) 
# X_test = scaler.transform(X_test)

print(len(X_train[0]))
# print(X_train)
# print('\n', X_test)

21


In [19]:
# Архитектура сети
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# for _ in range(3):
#     model.add(Dense(512, activation='relu', ))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.1))
# model.add(Dense(y_train.shape[1], activation='softmax'))

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))  # , kernel_regularizer=l2(0.001)
model.add(BatchNormalization())
model.add(Dropout(0.3))
for _ in range(5):
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.001)))  # , kernel_regularizer=l2(0.001)
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Параметры модели
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

c:\Users\Alkor\PycharmProjects\pj1_ML_classifier_RTS_SPAN_CC\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Определение функции обратного вызова EarlyStopping
early_stopping = EarlyStopping(
    # monitor='val_loss',      # Мониторим валидационную потерю
    monitor='val_accuracy',  # Мониторим валидационную точность
    patience=2000,           # Количество эпох без улучшения до остановки
    verbose=0,               # Вывод информации в процессе обучения
    restore_best_weights=True  # Восстанавливаем веса модели с наилучшей валидационной потерей
)

# Определение функции обратного вызова ModelCheckpoint
model_checkpoint = ModelCheckpoint(
    filepath='best_model.keras',   # Путь для сохранения модели
    # monitor='val_loss',            # Мониторим валидационную потерю
    monitor='val_accuracy',        # Мониторим валидационную точность
    save_best_only=True,           # Сохраняем только лучшую модель
    verbose=1                      # Вывод информации в процессе сохранения
)

# Обучение модели с использованием функций обратного вызова
history = model.fit(
    X_train, y_train,
    epochs=10000,               # Максимальное количество эпох
    batch_size=512,            # Размер мини-пакета
    validation_split=0.15,      # Доля данных для валидации
    callbacks=[early_stopping, model_checkpoint]  # Список функций обратного вызова
)


Epoch 1/10000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4763 - loss: 3.8460
Epoch 1: val_accuracy improved from -inf to 0.46980, saving model to best_model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.4775 - loss: 3.8414 - val_accuracy: 0.4698 - val_loss: 3.2926
Epoch 2/10000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5082 - loss: 3.7141
Epoch 2: val_accuracy improved from 0.46980 to 0.48993, saving model to best_model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5080 - loss: 3.7155 - val_accuracy: 0.4899 - val_loss: 3.2914
Epoch 3/10000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4871 - loss: 3.7266
Epoch 3: val_accuracy improved from 0.48993 to 0.49664, saving model to best_model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4893 - loss: 3.7224 - val_accuracy: 0.4966 - val_loss: 3.2902
Epoch 4/10000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5047 - loss: 3.7091
Epoch 4: val_accuracy did not improve from 0.4966

In [21]:
del model  # deletes the existing model
model = load_model('best_model.keras')  # Загрузка сохраненной лучшей модели

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5569 - loss: 2.1771  
Test Accuracy: 0.5413


In [23]:
predictions = model.predict(X_test)

# Пример вывода вероятностей для примеров из тестовой выборки
print(predictions)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[9.92864490e-01 7.13553606e-03]
 [1.41354278e-03 9.98586535e-01]
 [7.61004269e-01 2.38995701e-01]
 [9.99540448e-01 4.59581643e-04]
 [8.38514924e-01 1.61485076e-01]
 [3.98109287e-01 6.01890683e-01]
 [9.11955833e-02 9.08804357e-01]
 [9.73437369e-01 2.65626460e-02]
 [9.95742381e-01 4.25754301e-03]
 [7.93883204e-01 2.06116766e-01]
 [9.46428776e-01 5.35712242e-02]
 [3.76340076e-02 9.62366045e-01]
 [9.76439118e-01 2.35608350e-02]
 [9.97059226e-01 2.94084242e-03]
 [9.61103141e-01 3.88968289e-02]
 [8.70786071e-01 1.29213944e-01]
 [4.97054487e-01 5.02945423e-01]
 [8.25976729e-01 1.74023300e-01]
 [6.79294765e-01 3.20705205e-01]
 [9.87121999e-01 1.28780641e-02]
 [9.63155627e-01 3.68443355e-02]
 [1.07561983e-02 9.89243746e-01]
 [9.96934533e-01 3.06541240e-03]
 [9.67944443e-01 3.20555493e-02]
 [9.99802887e-01 1.97142159e-04]
 [9.35257912e-01 6.47420511e-02]
 [9.84406233e-01 1.55937681e-02]
 [7.09715307e-01 2.90284663e-01]
 [4.25178371e-03 9.95748222e-01]
 [4

In [24]:
np.savetxt('nn_predictions.txt', predictions, fmt='%.4f')
# print(predictions, fmt='%.4f')
# print(predictions('{:,.2f}'.format))

In [34]:
border = 0.6  # Пороговое значение вероятности по предсказанию

def profit_loss(u_p, d_p, u_r, d_r):
    if u_p > border and u_r == 1.0:
        return 1
    elif d_p > border and d_r == 1.0:
        return 1
    else:
        return 0

df = pd.DataFrame(predictions, columns=['up_predict', 'down_predict'])
df_y = pd.DataFrame(y_test, columns=['up_real', 'down_real'])
df = pd.merge(df, df_y, left_index=True, right_index=True)  # , suffixes=('_predict', '_real')
df = df[(df.up_predict > border) | (df.down_predict > border)]  # Отсекаем неудовлетворяющие строки
df['profit_loss'] = df.apply(lambda x: profit_loss(x.up_predict, x.down_predict, x.up_real, x.down_real), axis=1)
print(df.to_string(max_rows=20, max_cols=22))
print(df.shape)

     up_predict  down_predict  up_real  down_real  profit_loss
0      0.992864      0.007136      1.0        0.0            1
1      0.001414      0.998587      1.0        0.0            0
2      0.761004      0.238996      0.0        1.0            0
3      0.999540      0.000460      1.0        0.0            1
4      0.838515      0.161485      0.0        1.0            0
5      0.398109      0.601891      1.0        0.0            0
6      0.091196      0.908804      1.0        0.0            0
7      0.973437      0.026563      1.0        0.0            1
8      0.995742      0.004258      1.0        0.0            1
9      0.793883      0.206117      1.0        0.0            1
..          ...           ...      ...        ...          ...
341    0.174915      0.825085      1.0        0.0            0
342    0.989335      0.010665      1.0        0.0            1
343    0.003482      0.996518      0.0        1.0            1
344    0.985906      0.014094      1.0        0.0      

In [35]:
rez = df.profit_loss.sum() / len(df.profit_loss)
print(rez)

0.5214723926380368
